# Python 知识

# 代码分析

## 类继承关系
```mermaid
classDiagram
    class Configured
    class PandasIndexer
    class ColumnGrouper
    class ArrayWrapper
    class Wrapping
    
    Configured <|-- ArrayWrapper
    PandasIndexer <|-- ArrayWrapper
    Configured <|-- Wrapping
    PandasIndexer <|-- Wrapping
    AttrResolver <|-- Wrapping

    ArrayWrapper o-- ColumnGrouper : has
    Wrapping o-- ArrayWrapper : has
```

## class ArrayWrapper(Configured, PandasIndexer)

### 使用方式

#### 创建模拟市场数据

In [5]:
import numpy as np
import pandas as pd
from vectorbt.base.array_wrapper import ArrayWrapper

# 创建30天的时间索引（交易日）
dates = pd.date_range('2024-01-01', periods=30, freq='D', name='date')

# 模拟三只股票的OHLCV数据
np.random.seed(42)  # 确保结果可复现
symbols = ['AAPL', 'GOOGL', 'MSFT']

# 构建多列数据：每只股票包含开高低收价格和成交量
columns = []
for symbol in symbols:
    for data_type in ['open', 'high', 'low', 'close', 'volume']:
        columns.append(f"{symbol}_{data_type}")

        # 生成模拟的市场数据
n_days, n_columns = len(dates), len(columns)
base_prices = {'AAPL': 150, 'GOOGL': 2800, 'MSFT': 300}
market_data = np.zeros((n_days, n_columns))
        
for i, symbol in enumerate(symbols):
    base_price = base_prices[symbol]
    # 模拟价格随机游走
    price_changes = np.random.normal(0, 0.02, n_days)  # 2%日波动率
    prices = base_price * np.exp(np.cumsum(price_changes))
    # OHLC数据生成
    col_start = i * 5
    market_data[:, col_start] = prices * (1 + np.random.normal(0, 0.001, n_days))  # open
    market_data[:, col_start+1] = prices * (1 + np.abs(np.random.normal(0, 0.01, n_days)))  # high
    market_data[:, col_start+2] = prices * (1 - np.abs(np.random.normal(0, 0.01, n_days)))  # low
    market_data[:, col_start+3] = prices  # close
    market_data[:, col_start+4] = np.random.lognormal(15, 0.5, n_days)  # volume

print(f"market_data的形状，行数{market_data.shape[0]}表示日期，列数{market_data.shape[1]}表示三支股票的OHLCV数据")


market_data的形状，行数30表示日期，列数15表示三支股票的OHLCV数据


#### 创建 ArrayWrapper

In [6]:
# 创建基础包装器，管理完整的市场数据
market_wrapper = ArrayWrapper(
    index=dates,
    columns=pd.Index(columns),
    ndim=2,
    freq='D'  # 指定日频数据
)

print(f"市场数据维度: {market_wrapper.shape}")
print(f"时间序列频率: {market_wrapper.freq}")
print(f"数据列数: {len(market_wrapper.columns)}")

# 包装原始数据为DataFrame
market_df = market_wrapper.wrap(market_data)
print("\n市场数据预览:")
print(market_df.head(3))

市场数据维度: (30, 15)
时间序列频率: 1 days 00:00:00
数据列数: 15

市场数据预览:
             AAPL_open   AAPL_high    AAPL_low  AAPL_close   AAPL_volume  \
date                                                                       
2024-01-01  151.406412  152.223506  151.350499  151.497569  4.854982e+06   
2024-01-02  151.359054  151.359705  149.615792  151.079213  2.074657e+06   
2024-01-03  153.046924  154.742224  151.974504  153.048989  6.592196e+06   

             GOOGL_open   GOOGL_high    GOOGL_low  GOOGL_close  GOOGL_volume  \
date                                                                           
2024-01-01  2815.823464  2830.128014  2791.760765  2814.062797  6.720255e+06   
2024-01-02  2831.200179  2865.806763  2830.377840  2833.629045  1.594497e+06   
2024-01-03  2792.357658  2822.018878  2781.235011  2795.351178  5.847834e+06   

             MSFT_open   MSFT_high    MSFT_low  MSFT_close   MSFT_volume  
date                                                                      
2024-01-0

#### 分组 regroup 和解析 resolve

In [8]:
# 创建股票分组：每只股票的5个数据列归为一组
stock_groups = []
for symbol in symbols:
    stock_groups.extend([symbol] * 5)  # 每只股票5列数据

# 创建分组包装器
grouped_wrapper = market_wrapper.regroup(stock_groups)
print(f"\n分组后维度: {grouped_wrapper.get_shape()}")  # (30天, 3只股票)
print(f"分组标签: {grouped_wrapper.get_columns().tolist()}")

# 解析分组视图：将分组视为实际的列
resolved_wrapper = grouped_wrapper.resolve()
print(f"解析后维度: {resolved_wrapper.shape}")
print(f"解析后列名: {resolved_wrapper.columns.tolist()}")


分组后维度: (30, 3)
分组标签: ['AAPL', 'GOOGL', 'MSFT']
解析后维度: (30, 3)
解析后列名: ['AAPL', 'GOOGL', 'MSFT']


#### 返回元数据 indexing_func_meta

In [20]:
# indexing_func_meta是索引操作的核心方法，返回详细的索引元数据
# 返回值：(新包装器, 行索引数组, 列索引数组, 未分组列索引数组)

# 基础用法：选择最近5天和前3列数据
meta_result = market_wrapper.indexing_func_meta(
    lambda x: x.iloc[-5:, :3]
)
new_wrapper, row_idxs, col_idxs, ungrouped_col_idxs = meta_result
print("\n=== indexing_func_meta基础示例 ===")
print(f"新包装器形状: {new_wrapper.shape}")
print(f"选择的行索引: {row_idxs}")  # 最后5行的索引数组
print(f"选择的列索引: {col_idxs}")  # 前3列的索引数组
print(f"未分组列索引: {ungrouped_col_idxs}")  # 与col_idxs相同（无分组）

# column_only_select模式：将DataFrame视为列的Series进行索引
new_wrapper_col, row_idxs_col, col_idxs_col, ungrouped_col_idxs_col = market_wrapper.indexing_func_meta(
    lambda x: x[['AAPL_close', 'GOOGL_close']],  # 选择两只股票的收盘价
    column_only_select=True  # 启用列选择模式
)
print("\n=== column_only_select模式 ===")
print(f"列选择模式形状: {new_wrapper_col.shape}")  # 保留所有行，只选择指定列
print(f"行索引范围: [{row_idxs_col[0]}, ..., {row_idxs_col[-1]}]")  # 所有行
print(f"选择的列索引: {col_idxs_col}")  # 对应选择的列
print(f"选择的列名: {new_wrapper_col.columns.tolist()}")

# 分组情况下的索引操作：展示group_select的威力
grouped_meta = grouped_wrapper.indexing_func_meta(
    lambda x: x.iloc[-10:],  # 选择最近10天
    group_select=False  # 不基于分组选择
)
grouped_new_wrapper, grouped_row_idxs, grouped_col_idxs, grouped_ungrouped = grouped_meta
print("\n=== 分组包装器索引（group_select=False）===")
print(f"分组索引后形状: {grouped_new_wrapper.shape}")  # (10天, 15列)
print(f"行索引范围: [{grouped_row_idxs[0]}, ..., {grouped_row_idxs[-1]}]")
print(f"列索引数量: {len(grouped_col_idxs)}")
print(f"未分组列索引数量: {len(grouped_ungrouped)}")

# 基于分组的选择：group_select=True
group_select_meta = grouped_wrapper.indexing_func_meta(
    lambda x: x[['AAPL', 'MSFT']],  # 选择两个股票分组
    group_select=True  # 启用分组选择模式
)
gs_wrapper, gs_row_idxs, gs_group_idxs, gs_ungrouped_cols = group_select_meta
print("\n=== 分组选择模式（group_select=True）===")
print(f"分组选择后形状: {gs_wrapper.shape}")  # (30天, 10列)原来两个组的所有列
print(f"选择的分组索引: {gs_group_idxs}")  # [0, 2]对应AAPL和MSFT分组
print(f"展开后的列数: {len(gs_ungrouped_cols)}")  # 展开分组后的实际列数
print(f"展开后的列名前5个: {gs_wrapper.columns[:5].tolist()}")


=== indexing_func_meta基础示例 ===
新包装器形状: (5, 3)
选择的行索引: [25 26 27 28 29]
选择的列索引: [0 1 2]
未分组列索引: [0 1 2]

=== column_only_select模式 ===
列选择模式形状: (30, 2)
行索引范围: [0, ..., 29]
选择的列索引: [3 8]
选择的列名: ['AAPL_close', 'GOOGL_close']

=== 分组包装器索引（group_select=False）===
分组索引后形状: (10, 15)
行索引范围: [20, ..., 29]
列索引数量: 15
未分组列索引数量: 15

=== 分组选择模式（group_select=True）===
分组选择后形状: (30, 10)
选择的分组索引: [0 2]
展开后的列数: 10
展开后的列名前5个: ['AAPL_open', 'AAPL_high', 'AAPL_low', 'AAPL_close', 'AAPL_volume']


#### 索引操作 indexing_func

In [9]:
# 标准索引：选择特定时间段和股票
recent_wrapper = market_wrapper.indexing_func(lambda x: x.iloc[-10:, :])  # 最近10天
print(f"\n最近10天数据维度: {recent_wrapper.shape}")

# 列选择模式：只选择特定类型的数据
close_price_cols = [col for col in columns if 'close' in col]
close_price_idxs = [columns.index(col) for col in close_price_cols]
close_wrapper = market_wrapper.indexing_func(
    lambda x: x.iloc[:, close_price_idxs]
)
print(f"收盘价数据维度: {close_wrapper.shape}")

# 分组选择：选择特定股票的所有数据
aapl_wrapper = grouped_wrapper.indexing_func(lambda x: x['AAPL'])
print(f"AAPL数据维度: {aapl_wrapper.shape}")
print(f"AAPL数据类型: {aapl_wrapper.ndim}维")


最近10天数据维度: (10, 15)
收盘价数据维度: (30, 3)
AAPL数据维度: (30, 5)
AAPL数据类型: 2维


#### 使用 wrap

In [11]:
# 计算简单移动平均线（SMA）
def calculate_sma(prices, window=5):
    return np.convolve(prices, np.ones(window)/window, mode='valid')

# 提取收盘价数据
close_prices = market_df[close_price_cols].values

# 为SMA创建新的包装器（时间序列长度减少）
sma_dates = dates[4:]  # 5日均线从第5天开始
sma_wrapper = ArrayWrapper(
    index=sma_dates,
    columns=pd.Index(symbols),  # 只保留股票名称
    ndim=2,
    freq='D'
)

# 计算并包装SMA数据
sma_data = np.column_stack([
    calculate_sma(close_prices[:, i], 5) for i in range(len(symbols))
])
sma_df = sma_wrapper.wrap(sma_data)
print("\n5日移动平均线:")
print(sma_df.tail(3))

# 计算每日收益率
returns = np.diff(close_prices, axis=0) / close_prices[:-1]
returns_wrapper = ArrayWrapper(
    index=dates[1:],  # 收益率从第2天开始
    columns=pd.Index(symbols),
    ndim=2,
    freq='D'
)
returns_df = returns_wrapper.wrap(returns)
print("\n每日收益率:")
print(returns_df.tail(3))


5日移动平均线:
                  AAPL        GOOGL        MSFT
date                                           
2024-01-28  137.654803  3055.079090  342.117927
2024-01-29  136.661519  3083.332689  342.612533
2024-01-30  135.814079  3143.315147  345.382377

每日收益率:
                AAPL     GOOGL      MSFT
date                                    
2024-01-28  0.007542  0.029497  0.015959
2024-01-29 -0.011941 -0.005279  0.012561
2024-01-30 -0.005817  0.055910  0.012646


#### 使用 wrap_reduced

In [ ]:
import numpy as np
import pandas as pd
from vectorbt.base.array_wrapper import ArrayWrapper, Wrapping
import warnings

# ========================================================================
# resolve_self 方法详解：智能对象状态管理
# ========================================================================

class AdvancedTechnicalIndicator(Wrapping):
    """
    高级技术指标类，演示 resolve_self 方法的各种参数使用场景
    """
    
    def __init__(self, prices=None, period=20, freq='D', **kwargs):
        """
        初始化高级技术指标
        Args:
            prices: 价格数据
            period: 计算周期
            freq: 时间频率
        """
        wrapper = kwargs.pop('wrapper', None)
        
        if wrapper is not None:
            # 这是replace调用，直接使用提供的wrapper
            super().__init__(wrapper, **kwargs)
            self._prices = kwargs.get('prices')
            self._period = kwargs.get('period', period)
            return
        
        # 正常初始化
        if prices is None:
            raise ValueError("prices is required")
        
        if isinstance(prices, np.ndarray):
            prices = pd.DataFrame(prices)
        elif isinstance(prices, pd.Series):
            prices = prices.to_frame()
        
        # 计算技术指标（简单移动平均）
        indicator_values = prices.rolling(window=period).mean()
        
        # 创建包装器，特别设置freq参数
        wrapper = ArrayWrapper.from_obj(indicator_values, freq=freq)
        
        # 存储配置
        kwargs.update({
            'prices': prices,
            'period': period,
            'freq': freq
        })
        
        super().__init__(wrapper, **kwargs)
        self._prices = prices
        self._period = period
    
    @property
    def writeable_attrs(self):
        return {'_prices', '_period'}
    
    def calculate_with_different_freq(self, new_freq='H', 
                                    enable_caching=True, 
                                    show_warnings=True,
                                    custom_params=None):
        """
        演示 resolve_self 各参数的完整使用场景
        
        Args:
            new_freq: 新的时间频率
            enable_caching: 是否启用缓存
            show_warnings: 是否显示警告
            custom_params: 自定义参数
        """
        
        # ================================================================
        # 1. cond_kwargs 参数：条件参数字典
        # ================================================================
        # 作用：传递可能影响对象状态的参数，如频率变化
        # 意义：这些参数会被检查，如果导致状态变化，会创建新对象
        
        cond_kwargs = {
            'freq': new_freq,           # 最重要：频率变化会触发对象重建
            'self': self,               # 自引用，允许方法内部访问当前对象
            'period': self._period,     # 计算周期
            'enable_advanced': True     # 自定义业务参数
        }
        
        if custom_params:
            cond_kwargs.update(custom_params)
        
        print(f\"\\n=== 1. cond_kwargs 参数演示 ===\")
        print(f\"原始频率: {self.wrapper.freq}\")\n        print(f\"新频率: {new_freq}\")\n        print(f\"条件参数: {list(cond_kwargs.keys())}\")\n        \n        # ================================================================\n        # 2. custom_arg_names 参数：自定义参数名称集合\n        # ================================================================\n        # 作用：指定哪些参数名不应该被自动处理\n        # 意义：防止某些特殊参数被错误地更新或覆盖\n        \n        custom_arg_names = {\n            'enable_advanced',     # 业务逻辑参数，不需要自动更新\n            'period'               # 周期参数，希望手动控制\n        }\n        \n        print(f\"\\n=== 2. custom_arg_names 参数演示 ===\")\n        print(f\"自定义参数名称: {custom_arg_names}\")\n        print(f\"这些参数不会被自动处理和更新\")\n        \n        # ================================================================\n        # 3. impacts_caching 参数：是否影响缓存\n        # ================================================================\n        # 作用：控制当对象状态发生变化时，是否禁用缓存\n        # 意义：状态变化的对象使用缓存可能导致错误结果\n        \n        impacts_caching = not enable_caching  # 如果不启用缓存，则影响缓存\n        \n        print(f\"\\n=== 3. impacts_caching 参数演示 ===\")\n        print(f\"启用缓存: {enable_caching}\")\n        print(f\"影响缓存: {impacts_caching}\")\n        print(f\"当impacts_caching=True且发生状态变化时，会设置use_caching=False\")\n        \n        # ================================================================\n        # 4. silence_warnings 参数：是否静默警告\n        # ================================================================\n        # 作用：控制是否显示关于对象状态变化的警告信息\n        # 意义：频率变化等操作会创建对象副本，可能影响性能\n        \n        silence_warnings = not show_warnings\n        \n        print(f\"\\n=== 4. silence_warnings 参数演示 ===\")\n        print(f\"显示警告: {show_warnings}\")\n        print(f\"静默警告: {silence_warnings}\")\n        print(f\"频率变化时会显示性能提示警告（如果未静默）\")\n        \n        # ================================================================\n        # 实际调用 resolve_self 方法\n        # ================================================================\n        \n        print(f\"\\n=== resolve_self 方法调用 ===\")\n        print(f\"调用前对象ID: {id(self)}\")\n        \n        # 捕获警告以便演示\n        with warnings.catch_warnings(record=True) as w:\n            warnings.simplefilter(\"always\")\n            \n            resolved_self = self.resolve_self(\n                cond_kwargs=cond_kwargs,\n                custom_arg_names=custom_arg_names,\n                impacts_caching=impacts_caching,\n                silence_warnings=silence_warnings\n            )\n            \n            print(f\"调用后对象ID: {id(resolved_self)}\")\n            print(f\"是否创建了新对象: {id(self) != id(resolved_self)}\")\n            print(f\"新对象频率: {resolved_self.wrapper.freq}\")\n            \n            # 显示捕获的警告\n            if w and show_warnings:\n                print(f\"\\n捕获的警告数量: {len(w)}\")\n                for warning in w:\n                    print(f\"警告内容: {warning.message}\")\n            elif not show_warnings:\n                print(f\"\\n警告已被静默，捕获的警告数量: {len(w)}\")\n        \n        return resolved_self

# 创建测试数据
print(\"=== 创建测试数据 ===\")
np.random.seed(42)
dates = pd.date_range('2024-01-01', periods=30, freq='D')
prices = pd.Series(100 + np.cumsum(np.random.normal(0, 1, 30)), index=dates, name='AAPL')
print(f\"原始价格数据形状: {prices.shape}\")\nprint(f\"原始数据频率: {prices.index.freq}\")


In [13]:
# 计算每只股票的总收益率（累积）
total_returns = np.prod(1 + returns, axis=0) - 1
total_returns_series = returns_wrapper.wrap_reduced(total_returns, name_or_index='total_return')
print("\n股票总收益率:")
print(total_returns_series)


股票总收益率:
AAPL    -0.115579
GOOGL    0.176168
MSFT     0.204072
Name: total_return, dtype: float64


#### 使用 to_timedelta

In [14]:
holding_days = np.array([1, 5, 10, 20])  # 持仓天数
holding_periods = market_wrapper.to_timedelta(holding_days, to_pd=True)
print("\n持仓时间:")
for days, period in zip(holding_days, holding_periods):
    print(f"{days}天 = {period}")


持仓时间:
1天 = 1 days 00:00:00
5天 = 5 days 00:00:00
10天 = 10 days 00:00:00
20天 = 20 days 00:00:00


#### 重新分组演示

In [ ]:
# 定义行业分组
sectors = {'AAPL': 'Technology', 'GOOGL': 'Technology', 'MSFT': 'Technology'}
sector_groups = [sectors[symbol] for symbol in symbols]

# 按行业重新分组收益率数据
sector_wrapper = returns_wrapper.regroup(sector_groups)
print(f"\n行业分组维度: {sector_wrapper.get_shape()}")  # (29天, 1个行业)
print(f"行业分组标签: {sector_wrapper.get_columns().tolist()}")

# 计算行业平均收益率
sector_returns = returns_df.groupby(sector_groups, axis=1).mean()
print("\n行业平均收益率:")
print(sector_returns.tail(3))

### `__init__`
```python
    def __init__(self,
                 index: tp.IndexLike,                           # 行索引，支持类似pandas索引的对象
                 columns: tp.IndexLike,                         # 列索引，支持类似pandas索引的对象
                 ndim: int,                      # 数组维度数量，通常为1（Series）或2（DataFrame）
                 freq: tp.Optional[tp.FrequencyLike] = None,    # 时间序列频率，可选
                 column_only_select: tp.Optional[bool] = None,  # 是否仅对列进行索引，可选
                 group_select: tp.Optional[bool] = None,        # 是否对分组进行索引，可选
                 grouped_ndim: tp.Optional[int] = None,         # 分组后的维度数量，可选
                 **kwargs) -> None:                             # 传递给ColumnGrouper的额外参数
        config = dict(
            index=index,
            columns=columns,
            ndim=ndim,
            freq=freq,
            column_only_select=column_only_select,
            group_select=group_select,
            grouped_ndim=grouped_ndim,
        )

        checks.assert_not_none(index)
        checks.assert_not_none(columns)
        checks.assert_not_none(ndim)
        
        if not isinstance(index, pd.Index):
            index = pd.Index(index) 
        if not isinstance(columns, pd.Index):
            columns = pd.Index(columns) 

        self._index = index 
        self._columns = columns
        self._ndim = ndim
        self._freq = freq
        self._column_only_select = column_only_select
        self._group_select = group_select
        self._grouper = ColumnGrouper(columns, **kwargs)
        self._grouped_ndim = grouped_ndim

        PandasIndexer.__init__(self) 
        Configured.__init__(self, **merge_dicts(config, self._grouper._config)) 
```

### settings['array_wrapper']
```python
array_wrapper=dict(
    column_only_select=False,  # 不仅选择列
    group_select=True,  # 启用组选择
    freq=None,  # 频率默认为None
    silence_warnings=False  # 不静默警告
)
```

### regroup
核心是：基于当前的 `self` 构建一个新的 `ArrayWrapper`：使用参数 `group_by` 重新选择 `self.columns` 来构建 `.grouper`。

当 
- `self.grouper.group_by` 是否和 `group_by` 不完全一致
- 且二者情况符合：`self.allow_enable, self.allow_disable, self.allow_modify`

使用 `self.__dict__` 和 `group_by` 重新构建一个 `ArrayWrapper` 返回。否则返回 `self`。
```python
def regroup(self: ArrayWrapperT, group_by: tp.GroupByLike, **kwargs) -> ArrayWrapperT:
    if self.grouper.is_grouping_changed(group_by=group_by):
        self.grouper.check_group_by(group_by=group_by)
        grouped_ndim = None
        if self.grouper.is_grouped(group_by=group_by):
            if not self.grouper.is_group_count_changed(group_by=group_by):
                grouped_ndim = self.grouped_ndim
        return self.replace(grouped_ndim=grouped_ndim, group_by=group_by, **kwargs)
    return self
```

### resolve
使用参数 `group_by` 重新选择 `self.columns` 来构建 `.grouper`，从而生成一个新的 `ArrayWrapper` 实例 `_self`

取出 `_self.grouper.group_by` 的去重元素作为 `columns`。

基于`self.__dict__`，使用 `columns` 和 `group_by = None` 重新构建一个 `ArrayWrapper` 实例返回。

```python
@cached_method
def resolve(self: ArrayWrapperT, group_by: tp.GroupByLike = None, **kwargs) -> ArrayWrapperT:
    _self = self.regroup(group_by=group_by, **kwargs)
    if _self.grouper.is_grouped():
        return _self.replace(
            columns=_self.grouper.get_columns(),
            ndim=_self.grouped_ndim,
            grouped_ndim=None,
            group_by=None
        )
    return _self
```

### wrap
首先：
- 使用参数 `group_by` 重新选择 `self.columns` 来构建 `.grouper`，从而生成一个新的 `ArrayWrapper` 实例
- 取出该新实例的 `.grouper.group_by` 的去重元素作为 `columns`。
- 基于`self.__dict__`，使用 `columns` 和 `group_by = None` 重新构建一个 `ArrayWrapper` 实例 `_self`。
  
`columns` = `_self.columns` ∪ `columns`，`index` = `_self.index` ∪ `index`

将 `arr` 转换为 `Series/DataFrame`，并设置 `index` 和 `columns`。
```python
def wrap(self,
            arr: tp.ArrayLike,
            index: tp.Optional[tp.IndexLike] = None,
            columns: tp.Optional[tp.IndexLike] = None,
            fillna: tp.Optional[tp.Scalar] = None,
            dtype: tp.Optional[tp.PandasDTypeLike] = None,
            group_by: tp.GroupByLike = None,
            to_timedelta: bool = False,
            to_index: bool = False,
            silence_warnings: tp.Optional[bool] = None) -> tp.SeriesFrame:

    from vectorbt._settings import settings
    array_wrapper_cfg = settings['array_wrapper']

    if silence_warnings is None:
        silence_warnings = array_wrapper_cfg['silence_warnings']

    _self = self.resolve(group_by=group_by)

    if index is None:
        index = _self.index
    if not isinstance(index, pd.Index):
        index = pd.Index(index)

    if columns is None:
        columns = _self.columns
    if not isinstance(columns, pd.Index):
        columns = pd.Index(columns)

    if len(columns) == 1:
        name = columns[0]
        if name == 0:  # was a Series before
            name = None
    else:
        name = None

    def _wrap(arr):
        arr = np.asarray(arr)
        checks.assert_ndim(arr, (1, 2))
        if fillna is not None:
            arr[pd.isnull(arr)] = fillna
        arr = reshape_fns.soft_to_ndim(arr, self.ndim)
        checks.assert_shape_equal(arr, index, axis=(0, 0))
        if arr.ndim == 2:
            checks.assert_shape_equal(arr, columns, axis=(1, 0))
        if arr.ndim == 1:
            return pd.Series(arr, index=index, name=name, dtype=dtype)
        if arr.ndim == 2:
            if arr.shape[1] == 1 and _self.ndim == 1:
                return pd.Series(arr[:, 0], index=index, name=name, dtype=dtype)
            return pd.DataFrame(arr, index=index, columns=columns, dtype=dtype)
        raise ValueError(f"{arr.ndim}-d input is not supported")

    out = _wrap(arr)
    if to_index:
        # Convert to index
        if checks.is_series(out):
            out = out.map(lambda x: self.index[x] if x != -1 else np.nan)
        else:
            out = out.applymap(lambda x: self.index[x] if x != -1 else np.nan)
    if to_timedelta:
        # Convert to timedelta
        out = self.to_timedelta(out, silence_warnings=silence_warnings)
    return out
```

### wrap_reduced
首先：
- 使用参数 `group_by` 重新选择 `self.columns` 来构建 `.grouper`，从而生成一个新的 `ArrayWrapper` 实例
- 取出该新实例的 `.grouper.group_by` 的去重元素作为 `columns`。
- 基于`self.__dict__`，使用 `columns` 和 `group_by = None` 重新构建一个 `ArrayWrapper` 实例 `_self` 返回。

`columns` = `_self.columns` ∪ `columns`

将 `arr` 转换为 `Series/DataFrame`，并设置 `index` 和 `columns`：
- `arr` 是标量
- `arr` 是一维
  - `_self` 是一维
    - 参数 `name_or_index` ——> `index`
    - `columns[0]` ——> `name`
  - `_self` 是二维：
    - `columns` ——> `index`
    - 参数 `name_or_index` ——> `name`
- `arr` 是二维：
  - `arr` 实际上只有一列并且 `_self` 是一维
    - 参数 `name_or_index` ——> `index`
    - `columns[0]` ——> `name`
  - 否则
    - 参数 `name_or_index` ——> `index`
    - `columns` ——> `columns`
```python
def wrap_reduced(self,
                    arr: tp.ArrayLike,
                    name_or_index: tp.NameIndex = None,
                    columns: tp.Optional[tp.IndexLike] = None,
                    fillna: tp.Optional[tp.Scalar] = None,
                    dtype: tp.Optional[tp.PandasDTypeLike] = None,
                    group_by: tp.GroupByLike = None,
                    to_timedelta: bool = False,
                    to_index: bool = False,
                    silence_warnings: tp.Optional[bool] = None) -> tp.MaybeSeriesFrame:

    from vectorbt._settings import settings
    array_wrapper_cfg = settings['array_wrapper']

    if silence_warnings is None:
        silence_warnings = array_wrapper_cfg['silence_warnings']

    checks.assert_not_none(self.ndim)
    _self = self.resolve(group_by=group_by)

    if columns is None:
        columns = _self.columns
    if not isinstance(columns, pd.Index):
        columns = pd.Index(columns)

    if to_index:
        if dtype is None:
            dtype = np.int64
        if fillna is None:
            fillna = -1

    def _wrap_reduced(arr):
        nonlocal name_or_index

        arr = np.asarray(arr)
        if fillna is not None:
            arr[pd.isnull(arr)] = fillna
        if arr.ndim == 0:
            return pd.Series(arr, dtype=dtype)[0]
        if arr.ndim == 1:
            if _self.ndim == 1:
                if arr.shape[0] == 1:
                    return pd.Series(arr, dtype=dtype)[0]
                sr_name = columns[0]
                if sr_name == 0:
                    sr_name = None
                if isinstance(name_or_index, str):
                    name_or_index = None
                return pd.Series(arr, index=name_or_index, name=sr_name, dtype=dtype)
            return pd.Series(arr, index=columns, name=name_or_index, dtype=dtype)
        if arr.ndim == 2:
            if arr.shape[1] == 1 and _self.ndim == 1:
                arr = reshape_fns.soft_to_ndim(arr, 1)
                sr_name = columns[0]
                if sr_name == 0: 
                        sr_name = None
                    if isinstance(name_or_index, str):
                        name_or_index = None
                    return pd.Series(arr, index=name_or_index, name=sr_name, dtype=dtype)
                if isinstance(name_or_index, str):
                    name_or_index = None
                return pd.DataFrame(arr, index=name_or_index, columns=columns, dtype=dtype)
            raise ValueError(f"{arr.ndim}-d input is not supported")

        out = _wrap_reduced(arr)
        if to_index:
            if checks.is_series(out):
                out = out.map(lambda x: self.index[x] if x != -1 else np.nan)
            elif checks.is_frame(out):
                out = out.applymap(lambda x: self.index[x] if x != -1 else np.nan)
            else:
                out = self.index[out] if out != -1 else np.nan
        if to_timedelta:
            out = self.to_timedelta(out, silence_warnings=silence_warnings)
        return out
```

### grouped_ndim
```python
@property
def grouped_ndim(self) -> int:
    if self._grouped_ndim is None:
        if self.grouper.is_grouped():
            return 2 if self.grouper.get_group_count() > 1 else 1
        return self.ndim
    return self._grouped_ndim
```

## class Wrapping(Configured, PandasIndexer, AttrResolver)

### 使用方式
被某类继承。

#### 创建 RSI 和 MA 指标类

In [3]:
import numpy as np
import pandas as pd
from vectorbt.base.array_wrapper import ArrayWrapper, Wrapping

# ========================================================================
# 1. 构建技术指标基类 - RSI相对强弱指数
# ========================================================================
class RSIIndicator(Wrapping):
    '''RSI相对强弱指数技术指标类'''
    
    def __init__(self, close_prices=None, period=14, **kwargs):
        '''
        初始化RSI指标
        Args:
            close_prices: 收盘价数据 (pandas Series或DataFrame)，可选
            period: RSI计算周期，默认14天
        '''
        # 从kwargs中提取wrapper（如果存在，说明是replace调用）
        wrapper = kwargs.pop('wrapper', None)
        
        if wrapper is not None:
            # 这是replace调用，wrapper已经提供，直接使用
            super().__init__(wrapper, **kwargs)
            
            # 从配置中恢复参数
            self._close_prices = kwargs.get('close_prices')
            self._period = kwargs.get('period', period)
            self._rsi_values = None  # 将在需要时重新计算
            return
        
        # 正常初始化流程
        if close_prices is None:
            raise ValueError("close_prices is required for RSI calculation")
        
        # 确保输入是pandas对象
        if isinstance(close_prices, np.ndarray):
            close_prices = pd.DataFrame(close_prices)
        elif isinstance(close_prices, pd.Series):
            close_prices = close_prices.to_frame()
        
        # 计算RSI指标
        rsi_values = self._calculate_rsi(close_prices, period)
        
        # 创建ArrayWrapper管理RSI数据
        wrapper = ArrayWrapper.from_obj(rsi_values, freq='D')
        
        # 将原始数据存储到配置中，以便replace方法可以访问
        kwargs['close_prices'] = close_prices
        kwargs['period'] = period
        
        super().__init__(wrapper, **kwargs)
        
        # 存储原始数据和参数作为实例属性
        self._close_prices = close_prices
        self._rsi_values = rsi_values
        self._period = period
    
    @property
    def writeable_attrs(self):
        """可写属性，确保replace时正确复制"""
        return {'_close_prices', '_rsi_values', '_period'}
    
    def _calculate_rsi(self, prices, period):
        '''计算RSI值'''
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi.fillna(50)  # 用50填充初始NaN值
    
    @property
    def values(self):
        '''获取RSI值'''
        if self._rsi_values is None:
            # 如果RSI值不存在，尝试从wrapper重建
            # 这种情况发生在replace调用后
            if hasattr(self, '_close_prices') and self._close_prices is not None:
                # 这里我们不能重新计算RSI，因为数据可能已经被索引了
                # 应该让调用方自己处理
                raise ValueError("RSI values not available after indexing. Please recalculate.")
        return self._rsi_values
    
    @property
    def overbought_signals(self):
        '''RSI超买信号 (>70)'''
        return self.values > 70
    
    @property
    def oversold_signals(self):
        '''RSI超卖信号 (<30)'''
        return self.values < 30
    
    def get_signals(self, overbought_threshold=70, oversold_threshold=30):
        '''获取交易信号'''
        values = self.values
        signals = pd.DataFrame(index=values.index, columns=values.columns)
        signals[values > overbought_threshold] = 'SELL'
        signals[values < oversold_threshold] = 'BUY'
        signals = signals.fillna('HOLD')
        return signals

# ========================================================================
# 2. 构建移动平均线指标类 - 展示多时间周期分析
# ========================================================================   
class MovingAverageIndicator(Wrapping):
    '''移动平均线技术指标类'''
    
    def __init__(self, prices=None, short_window=5, long_window=20, **kwargs):
        '''
        初始化移动平均线指标
        Args:
            prices: 价格数据，可选
            short_window: 短期均线周期
            long_window: 长期均线周期
        '''
        # 从kwargs中提取wrapper（如果存在，说明是replace调用）
        wrapper = kwargs.pop('wrapper', None)
        
        if wrapper is not None:
            # 这是replace调用，wrapper已经提供，直接使用
            super().__init__(wrapper, **kwargs)
            
            # 从配置中恢复参数
            self._prices = kwargs.get('prices')
            self._short_window = kwargs.get('short_window', short_window)
            self._long_window = kwargs.get('long_window', long_window)
            self._combined_data = None  # 将在需要时重新计算或通过wrapper获取
            return
        
        # 正常初始化流程
        if prices is None:
            raise ValueError("prices is required for Moving Average calculation")
        
        if isinstance(prices, np.ndarray):
            prices = pd.DataFrame(prices)
        elif isinstance(prices, pd.Series):
            prices = prices.to_frame()
        
        # 计算短期和长期移动平均线
        short_ma = prices.rolling(window=short_window).mean()
        long_ma = prices.rolling(window=long_window).mean()
        
        # 组合成多列数据：原价格、短期MA、长期MA
        combined_data = pd.concat([
            prices.add_suffix('_price'),
            short_ma.add_suffix(f'_ma{short_window}'),
            long_ma.add_suffix(f'_ma{long_window}')
        ], axis=1)
        
        # 创建包装器
        wrapper = ArrayWrapper.from_obj(combined_data, freq='D')
        
        # 将数据存储到配置中
        kwargs.update({
            'prices': prices,
            'short_window': short_window,
            'long_window': long_window
        })
        
        super().__init__(wrapper, **kwargs)
        
        self._prices = prices
        self._combined_data = combined_data
        self._short_window = short_window
        self._long_window = long_window
    
    @property
    def writeable_attrs(self):
        """可写属性，确保replace时正确复制"""
        return {'_prices', '_combined_data', '_short_window', '_long_window'}
    
    @property
    def combined_data(self):
        '''获取组合数据（价格+均线）'''
        if self._combined_data is None:
            # 如果组合数据不存在，说明是replace调用后
            # 在实际应用中，这里应该通过wrapper重建数据或抛出适当的错误
            raise ValueError("Combined data not available after indexing. Data follows wrapper.")
        return self._combined_data
    
    @property
    def golden_cross(self):
        '''金叉信号：短期均线上穿长期均线'''
        data = self.combined_data
        short_cols = [col for col in data.columns if f'_ma{self._short_window}' in col]
        long_cols = [col for col in data.columns if f'_ma{self._long_window}' in col]
        
        signals = pd.DataFrame(index=data.index)
        for short_col, long_col in zip(short_cols, long_cols):
            symbol = short_col.replace(f'_ma{self._short_window}', '')
            short_ma = data[short_col]
            long_ma = data[long_col]
            # 检测金叉：短期均线从下方穿越长期均线
            signals[symbol] = ((short_ma > long_ma) & 
                              (short_ma.shift(1) <= long_ma.shift(1)))
        return signals
    
    @property  
    def death_cross(self):
        '''死叉信号：短期均线下穿长期均线'''
        data = self.combined_data
        short_cols = [col for col in data.columns if f'_ma{self._short_window}' in col]
        long_cols = [col for col in data.columns if f'_ma{self._long_window}' in col]
        
        signals = pd.DataFrame(index=data.index)
        for short_col, long_col in zip(short_cols, long_cols):
            symbol = short_col.replace(f'_ma{self._short_window}', '')
            short_ma = data[short_col]
            long_ma = data[long_col]
            # 检测死叉：短期均线从上方穿越长期均线
            signals[symbol] = ((short_ma < long_ma) & 
                              (short_ma.shift(1) >= long_ma.shift(1)))
        return signals

#### 创建模拟的多股票价格数据

In [4]:
# 创建模拟的多股票价格数据
np.random.seed(42)
dates = pd.date_range('2024-01-01', periods=60, freq='D')
symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA']

# 生成更真实的价格走势
price_data = {}
base_prices = {'AAPL': 150, 'GOOGL': 2800, 'MSFT': 300, 'TSLA': 200}
for symbol in symbols:
    base = base_prices[symbol]
    returns = np.random.normal(0.001, 0.02, len(dates))  # 日收益率
    prices = base * np.exp(np.cumsum(returns))
    price_data[symbol] = prices

market_prices = pd.DataFrame(price_data, index=dates)
print("\n=== 模拟市场数据 ===")
print(f"数据维度: {market_prices.shape}")
print(f"股票代码: {market_prices.columns.tolist()}")
print(f"时间范围: {market_prices.index[0]} 到 {market_prices.index[-1]}")
print("\n前5天价格:")
print(market_prices.head())


=== 模拟市场数据 ===
数据维度: (60, 4)
股票代码: ['AAPL', 'GOOGL', 'MSFT', 'TSLA']
时间范围: 2024-01-01 00:00:00 到 2024-02-29 00:00:00

前5天价格:
                  AAPL        GOOGL        MSFT        TSLA
2024-01-01  151.649142  2776.069095  305.088871  202.721013
2024-01-02  151.381674  2768.547330  299.889887  199.474730
2024-01-03  153.508826  2710.670587  308.731279  195.443183
2024-01-04  158.415077  2649.237654  300.495936  197.535667
2024-01-05  157.832697  2695.334829  304.347875  196.851553


#### RSIIndicator 使用 indexing_func、regroup、select_one

In [5]:
# 创建RSI指标实例
rsi_indicator = RSIIndicator(market_prices, period=14)
print("\n=== RSI指标分析 ===")
print(f"RSI指标数据形状: {rsi_indicator.wrapper.shape}")
print(f"RSI指标时间频率: {rsi_indicator.wrapper.freq}")
print(f"包装器类型: {type(rsi_indicator.wrapper)}")

# 获取最新的RSI值
latest_rsi = rsi_indicator.values.iloc[-1]
print("\n最新RSI值:")
for symbol, rsi_val in latest_rsi.items():
    status = "超买" if rsi_val > 70 else "超卖" if rsi_val < 30 else "正常"
    print(f"{symbol}: {rsi_val:.2f} ({status})")

# 展示Wrapping的索引功能：选择最近10天的数据
recent_rsi = rsi_indicator.indexing_func(lambda x: x.iloc[-10:])
print(f"\n最近10天RSI数据形状: {recent_rsi.wrapper.shape}")
print("最近10天RSI均值:")
print(recent_rsi.values.mean().round(2))

# ========================================================================
# 5. 分组分析功能 - 按行业分组进行技术分析
# ========================================================================

# 定义行业分组
sector_groups = ['Tech', 'Tech', 'Tech', 'Auto']  # AAPL, GOOGL, MSFT, TSLA
grouped_rsi = rsi_indicator.regroup(sector_groups)
print("\n=== 行业分组RSI分析 ===")
print(f"分组后形状: {grouped_rsi.wrapper.get_shape()}")
print(f"分组标签: {grouped_rsi.wrapper.get_columns().tolist()}")
print(f"是否已分组: {grouped_rsi.wrapper.grouper.is_grouped()}")

# 按行业计算平均RSI
sector_avg_rsi = grouped_rsi.values.groupby(sector_groups, axis=1).mean()
print("\n行业平均RSI（最近5天）:")
print(sector_avg_rsi.tail())

# 从多股票RSI中选择单个股票进行深入分析
aapl_rsi = rsi_indicator.select_one('AAPL')
print("\n=== 单股票深入分析 ===")
print(f"AAPL RSI数据维度: {aapl_rsi.wrapper.shape}")
print(f"AAPL RSI名称: {aapl_rsi.wrapper.name}")

# 从分组后的数据中选择科技行业
tech_rsi = grouped_rsi.select_one('Tech')
print("\n=== 科技行业RSI分析 ===")
print(f"科技行业RSI形状: {tech_rsi.wrapper.shape}")
tech_avg = tech_rsi.values.mean(axis=1)  # 计算科技行业平均RSI
print(f"科技行业当前平均RSI: {tech_avg.iloc[-1]:.2f}")


=== RSI指标分析 ===
RSI指标数据形状: (60, 4)
RSI指标时间频率: 1 days 00:00:00
包装器类型: <class 'vectorbt.base.array_wrapper.ArrayWrapper'>

最新RSI值:
AAPL: 59.35 (正常)
GOOGL: 70.35 (超买)
MSFT: 76.45 (超买)
TSLA: 48.41 (正常)

最近10天RSI数据形状: (10, 4)
最近10天RSI均值:
AAPL     38.33
GOOGL    50.21
MSFT     53.92
TSLA     54.82
dtype: float64

=== 行业分组RSI分析 ===
分组后形状: (60, 2)
分组标签: ['Tech', 'Auto']
是否已分组: True

行业平均RSI（最近5天）:
                 Auto       Tech
2024-02-25  54.681659  59.531573
2024-02-26  42.891390  57.977291
2024-02-27  50.401179  60.378802
2024-02-28  49.539458  63.383691
2024-02-29  48.413554  68.714358

=== 单股票深入分析 ===
AAPL RSI数据维度: (60,)
AAPL RSI名称: AAPL

=== 科技行业RSI分析 ===
科技行业RSI形状: (60, 3)
科技行业当前平均RSI: 63.64


C:\Users\shiguangjian5\AppData\Local\Temp\ipykernel_14116\764175469.py:34: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  sector_avg_rsi = grouped_rsi.values.groupby(sector_groups, axis=1).mean()


#### MovingAverageIndicator 使用 indexing_func

In [72]:
# 创建移动平均线指标
ma_indicator = MovingAverageIndicator(market_prices, short_window=5, long_window=20)
print("\n=== 移动平均线指标分析 ===")
print(f"MA指标数据形状: {ma_indicator.wrapper.shape}")  # 应该包含价格+短期MA+长期MA
print(f"数据列数: {len(ma_indicator.wrapper.columns)}")  # 4股票 × 3类型 = 12列

# 检测金叉信号
golden_crosses = ma_indicator.golden_cross
recent_golden = golden_crosses.tail(5)
print("\n最近5天金叉信号:")
print(recent_golden)

# 统计每只股票的金叉次数
golden_counts = golden_crosses.sum()
print("\n各股票金叉信号总次数:")
for symbol, count in golden_counts.items():
    print(f"{symbol}: {count}次")

# ========================================================================
# 高级索引操作 - column_only_select和时间片段分析
# ========================================================================

# 使用column_only_select选择特定股票
aapl_ma = ma_indicator.indexing_func(
    lambda x: x[['AAPL_price', 'AAPL_ma5', 'AAPL_ma20']],
    column_only_select=True
)
print("\n=== AAPL单股票MA分析 ===")
print(f"AAPL MA数据形状: {aapl_ma.wrapper.shape}")
print("AAPL最新数据:")
print(aapl_ma.wrapper.wrap(np.zeros(aapl_ma.wrapper.shape)).iloc[-1:])

# 时间切片：选择特定月份的数据
jan_data = ma_indicator.indexing_func(
    lambda x: x.loc[x.index.month == 1]  # 选择1月份数据
)
print(f"\n1月份数据形状: {jan_data.wrapper.shape}")



=== 移动平均线指标分析 ===
MA指标数据形状: (60, 12)
数据列数: 12

最近5天金叉信号:
             AAPL  GOOGL   MSFT   TSLA
2024-02-25  False  False  False  False
2024-02-26  False  False  False  False
2024-02-27  False  False  False   True
2024-02-28   True  False  False  False
2024-02-29  False  False  False  False

各股票金叉信号总次数:
AAPL: 1次
GOOGL: 2次
MSFT: 1次
TSLA: 2次

=== AAPL单股票MA分析 ===
AAPL MA数据形状: (60, 3)
AAPL最新数据:
            AAPL_price  AAPL_ma5  AAPL_ma20
2024-02-29         0.0       0.0        0.0

1月份数据形状: (31, 12)


In [73]:
# ========================================================================
# 10. 综合分析：构建交易策略信号
# ========================================================================

# 结合RSI和MA信号生成综合交易策略
def generate_trading_signals(rsi_indicator, ma_indicator):
    '''生成综合交易信号'''
    signals = pd.DataFrame(index=rsi_indicator.values.index,
                          columns=rsi_indicator.values.columns)
    
    # 获取信号
    rsi_oversold = rsi_indicator.oversold_signals  # RSI超卖
    rsi_overbought = rsi_indicator.overbought_signals  # RSI超买
    golden_cross = ma_indicator.golden_cross  # 金叉
    death_cross = ma_indicator.death_cross  # 死叉
    
    # 综合信号逻辑
    for symbol in signals.columns:
        # 强买信号：RSI超卖 + 金叉
        strong_buy = rsi_oversold[symbol] & golden_cross[symbol]
        # 强卖信号：RSI超买 + 死叉  
        strong_sell = rsi_overbought[symbol] & death_cross[symbol]
        # 一般买入：仅金叉
        buy = golden_cross[symbol] & ~strong_buy
        # 一般卖出：仅死叉
        sell = death_cross[symbol] & ~strong_sell
        
        signals.loc[strong_buy, symbol] = 'STRONG_BUY'
        signals.loc[strong_sell, symbol] = 'STRONG_SELL'
        signals.loc[buy, symbol] = 'BUY'
        signals.loc[sell, symbol] = 'SELL'
    
    return signals.fillna('HOLD')

trading_signals = generate_trading_signals(rsi_indicator, ma_indicator)
print("\n=== 综合交易信号分析 ===")
print("最近5天交易信号:")
print(trading_signals.tail())

# 统计各类信号的出现次数
signal_counts = {}
for signal in ['STRONG_BUY', 'BUY', 'HOLD', 'SELL', 'STRONG_SELL']:
    count = (trading_signals == signal).sum().sum()
    signal_counts[signal] = count
print("\n交易信号统计:")
for signal, count in signal_counts.items():
    print(f"{signal}: {count}次")


=== 综合交易信号分析 ===
最近5天交易信号:
            AAPL GOOGL  MSFT  TSLA
2024-02-25  HOLD  HOLD  HOLD  HOLD
2024-02-26  HOLD  HOLD  HOLD  HOLD
2024-02-27  HOLD  HOLD  HOLD   BUY
2024-02-28   BUY  HOLD  HOLD  HOLD
2024-02-29  HOLD  HOLD  HOLD  HOLD

交易信号统计:
STRONG_BUY: 0次
BUY: 6次
HOLD: 231次
SELL: 3次
STRONG_SELL: 0次


### `__init__`
```python
def __init__(self, wrapper: ArrayWrapper, **kwargs) -> None:
    checks.assert_instance_of(wrapper, ArrayWrapper)
    self._wrapper = wrapper  # 存储包装器实例

    # 按照MRO顺序初始化父类
    Configured.__init__(self, wrapper=wrapper, **kwargs)
    PandasIndexer.__init__(self)
    AttrResolver.__init__(self)
```

### indexing_func
调用 `self.wrapper.indexing_func` 返回一个新的 `ArrayWrapper` 实例。

然后调用 `self.replace`（父类 `Configured` 实现的）创建一个新的 `type(self)` 实例。

```python
def indexing_func(self: WrappingT, pd_indexing_func: tp.PandasIndexingFunc, **kwargs) -> WrappingT:
    return self.replace(wrapper=self.wrapper.indexing_func(pd_indexing_func, **kwargs))
```

### wrapper
```python
@property
def wrapper(self) -> ArrayWrapper:
    return self._wrapper
```

### regroup
基于 `self.wrapper` 构建一个新的 `ArrayWrapper`：使用参数 `group_by` 重新选择 `self.wrapper.columns` 来构建 `self.wrapper.grouper`。

然后调用 `self.replace`（父类 `Configured` 实现的）创建一个新的 `type(self)` 实例。
```python
def regroup(self: WrappingT, group_by: tp.GroupByLike, **kwargs) -> WrappingT:
    if self.wrapper.grouper.is_grouping_changed(group_by=group_by):
        self.wrapper.grouper.check_group_by(group_by=group_by)
        return self.replace(wrapper=self.wrapper.regroup(group_by, **kwargs))
    return self  # important for keeping cache
```

### resolve_self
```python
def resolve_self(self: AttrResolverT,
                    cond_kwargs: tp.KwargsLike = None,
                    custom_arg_names: tp.Optional[tp.Set[str]] = None,
                    impacts_caching: bool = True,
                    silence_warnings: tp.Optional[bool] = None) -> AttrResolverT:
    from vectorbt._settings import settings
    array_wrapper_cfg = settings['array_wrapper']

    if cond_kwargs is None:
        cond_kwargs = {}
    if custom_arg_names is None:
        custom_arg_names = set()
    if silence_warnings is None:
        silence_warnings = array_wrapper_cfg['silence_warnings']

    # 检查频率参数是否发生变化
    if 'freq' in cond_kwargs:
        wrapper_copy = self.wrapper.replace(freq=cond_kwargs['freq'])

        if wrapper_copy.freq != self.wrapper.freq:
            if not silence_warnings:
                warnings.warn(f"Changing the frequency will create a copy of this object. "
                                f"Consider setting it upon object creation to re-use existing cache.", stacklevel=2)
            self_copy = self.replace(wrapper=wrapper_copy)
            # 更新条件参数中的自引用
            for alias in self.self_aliases:
                if alias not in custom_arg_names:
                    cond_kwargs[alias] = self_copy
            cond_kwargs['freq'] = self_copy.wrapper.freq
            if impacts_caching:
                cond_kwargs['use_caching'] = False
            return self_copy
    return self
```

### select_one
```python
def select_one(self: WrappingT, column: tp.Any = None, group_by: tp.GroupByLike = None, **kwargs) -> WrappingT:
    _self = self.regroup(group_by, **kwargs)

    def _check_out_dim(out: WrappingT) -> WrappingT:
        """检查输出维度是否正确"""
        if _self.wrapper.grouper.is_grouped():
            if out.wrapper.grouped_ndim != 1:
                raise TypeError("Could not select one group: multiple groups returned")
        else:
            if out.wrapper.ndim != 1:
                raise TypeError("Could not select one column: multiple columns returned")
        return out

    if column is not None:
        # 指定了要选择的列/组
        if _self.wrapper.grouper.is_grouped():
            # 分组模式
            if _self.wrapper.grouped_ndim == 1:
                raise TypeError("This object already contains one group of data")
            if column not in _self.wrapper.get_columns():
                if isinstance(column, int):
                    # 尝试按位置索引
                    if _self.wrapper.column_only_select:
                        return _check_out_dim(_self.iloc[column])
                    return _check_out_dim(_self.iloc[:, column])
                raise KeyError(f"Group '{column}' not found")
        else:
            # 非分组模式
            if _self.wrapper.ndim == 1:
                raise TypeError("This object already contains one column of data")
            if column not in _self.wrapper.columns:
                if isinstance(column, int):
                    # 尝试按位置索引
                    if _self.wrapper.column_only_select:
                        return _check_out_dim(_self.iloc[column])
                    return _check_out_dim(_self.iloc[:, column])
                raise KeyError(f"Column '{column}' not found")
        return _check_out_dim(_self[column])
    
    # 未指定column，根据当前状态自动处理
    if not _self.wrapper.grouper.is_grouped():
        # 非分组模式
        if _self.wrapper.ndim == 1:
            return _self  # 已经是单列，直接返回
        raise TypeError("Only one column is allowed. Use indexing or column argument.")
    
    # 分组模式
    if _self.wrapper.grouped_ndim == 1:
        return _self  # 已经是单组，直接返回
    raise TypeError("Only one group is allowed. Use indexing or column argument.")
```

### select_one_from_obj
```python
@staticmethod
def select_one_from_obj(obj: tp.SeriesFrame, wrapper: ArrayWrapper, column: tp.Any = None) -> tp.MaybeSeries:
    if column is not None:
        # 指定了要选择的列/组
        if wrapper.ndim == 1:
            raise TypeError("This object already contains one column of data")
        
        if wrapper.grouper.is_grouped():
            # 分组模式
            if column not in wrapper.get_columns():
                if isinstance(column, int):
                    # 按位置选择
                    if isinstance(obj, pd.DataFrame):
                        return obj.iloc[:, column]
                    return obj.iloc[column]
                raise KeyError(f"Group '{column}' not found")
        else:
            # 非分组模式
            if column not in wrapper.columns:
                if isinstance(column, int):
                    # 按位置选择
                    if isinstance(obj, pd.DataFrame):
                        return obj.iloc[:, column]
                    return obj.iloc[column]
                raise KeyError(f"Column '{column}' not found")
        return obj[column]
    
    # 未指定column，根据当前状态自动处理
    if not wrapper.grouper.is_grouped():
        # 非分组模式
        if wrapper.ndim == 1:
            return obj  # 已经是单列，直接返回
        raise TypeError("Only one column is allowed. Use indexing or column argument.")
    
    # 分组模式
    if wrapper.grouped_ndim == 1:
        return obj  # 已经是单组，直接返回
    raise TypeError("Only one group is allowed. Use indexing or column argument.")
``` 